In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import re, string, unicodedata
import os
import time
start_time = time.time()
print(os.listdir("../input"))

In [ ]:
# Data Import
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
# Shape for train and test
print('Shape of train:',train.shape)
print('Shape of test:',test.shape)
print("--- %s seconds for Data Loading ---" % (time.time() - start_time))

In [ ]:
train["num_words"] = train["question_text"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["question_text"].apply(lambda x: len(str(x).split()))
#### Stats of Number of Words
print('maximum of num_words in train',train["num_words"].max())
print('min of num_words in train',train["num_words"].min())
print("maximum of  num_words in test",test["num_words"].max())
print('min of num_words in test',test["num_words"].min())

In [ ]:
train["num_unique_words"] = train["question_text"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["question_text"].apply(lambda x: len(set(str(x).split())))
#### Stats of Number of Unique Words
print('maximum of num_unique_words in train',train["num_unique_words"].max())
print('mean of num_unique_words in train',train["num_unique_words"].mean())
print("maximum of num_unique_words in test",test["num_unique_words"].max())
print('mean of num_unique_words in test',test["num_unique_words"].mean())

In [ ]:
train["num_chars"] = train["question_text"].apply(lambda x: len(str(x)))
test["num_chars"] = test["question_text"].apply(lambda x: len(str(x)))
#### Stats of Number of Characters
print('maximum of num_chars in train',train["num_chars"].max())
print("maximum of num_chars in test",test["num_chars"].max())

In [ ]:
train["num_punctuations"] =train['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_punctuations"] =test['question_text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
#### Stats of Number of Punctuations
print('maximum of num_punctuations in train',train["num_punctuations"].max())
print("maximum of num_punctuations in test",test["num_punctuations"].max())

In [ ]:
train["num_words_upper"] = train["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_upper"] = test["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
#### Stats of Number of Words in Uppercase
print('maximum of num_words_upper in train',train["num_words_upper"].max())
print("maximum of num_words_upper in test",test["num_words_upper"].max())

In [ ]:
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

cnt_srs = train['target'].value_counts()
## target distribution ##
labels = (np.array(cnt_srs.index))
sizes = (np.array((cnt_srs / cnt_srs.sum())*100))

trace = go.Pie(labels=labels, values=sizes)
layout = go.Layout(
    title='Target distribution',
    font=dict(size=12),
    width=500,
    height=500,
)
data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="usertype")


# Boxplots

In [ ]:
import matplotlib
from matplotlib import pyplot  as plt



fig, ax = plt.subplots(figsize=(15,2))
sns.boxplot(x="num_words", y="target", data=train, ax=ax, orient='h')
ax.set_xlabel('Word Count', size=10)
ax.set_ylabel('Target', size=10)
ax.set_title('Word Count distribution', size=12)
plt.gca().xaxis.grid(True)
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot  as plt


fig, ax = plt.subplots(figsize=(15,2))
sns.boxplot(x="num_unique_words", y="target", data=train, ax=ax, orient='h')
ax.set_xlabel('Unique word count', size=10)
ax.set_ylabel('Target', size=10)
ax.set_title('Unique word count distribution', size=12)
plt.gca().xaxis.grid(True)
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot  as plt



fig, ax = plt.subplots(figsize=(15,2))
sns.boxplot(x="num_chars", y="target", data=train, ax=ax, orient='h')
ax.set_xlabel('Character Count', size=10)
ax.set_ylabel('Target', size=10)
ax.set_title('Character count distribution', size=12)
plt.gca().xaxis.grid(True)
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot  as plt


fig, ax = plt.subplots(figsize=(15,2))
sns.boxplot(x="num_punctuations", y="target", data=train, ax=ax, orient='h')
ax.set_xlabel('Punctuation Count', size=10)
ax.set_ylabel('Target', size=10)
ax.set_title('Punctuation count distribution', size=12)
plt.gca().xaxis.grid(True)
plt.show()

In [ ]:
import matplotlib
from matplotlib import pyplot  as plt


fig, ax = plt.subplots(figsize=(15,2))
sns.boxplot(x="num_words_upper", y="target", data=train, ax=ax, orient='h')
ax.set_xlabel('Upper case count', size=10)
ax.set_ylabel('Target', size=10)
ax.set_title('Upper case count distribution', size=12)
plt.gca().xaxis.grid(True)
plt.show()

# Wordclouds

In [ ]:
from wordcloud import WordCloud, STOPWORDS

def generate_wordcloud(text): 
    wordcloud = WordCloud(relative_scaling = 1.0,stopwords = set(STOPWORDS), max_words= 15, background_color ='white').generate(text)
    fig,ax = plt.subplots(1,1,figsize=(10,10))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis("off")
    ax.margins(x=0, y=0)
    plt.show()

In [ ]:
# Sincere
generate_wordcloud(" ".join(train.question_text[train.target == 0]))

In [ ]:
# Insincere
generate_wordcloud(" ".join(train.question_text[train.target == 1]))

Weights

In [ ]:
train['Dataset'] = 'train'
test['Dataset'] = 'test'

all_data =  pd.concat([train, test], axis= 0, ignore_index= True)

import nltk
#!pip install inflect #Make sure the Kernal has Internet Connected (Check Settings)
#import inflect
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer 

#lower case
all_data['question_text'] = all_data['question_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation
all_data['question_text'] = all_data['question_text'].str.replace('[^\w\s]','')
#Removing numbers
all_data['question_text'] = all_data['question_text'].str.replace('[0-9]','')
#Remooving stop words and words with length <=2
from nltk.corpus import stopwords
stop = stopwords.words('english')
all_data['question_text'] = all_data['question_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop and len(x)>2))
# Lemmatize
from nltk.stem import WordNetLemmatizer
wl = WordNetLemmatizer()
all_data['question_text'] = all_data['question_text'].apply(lambda x: " ".join(wl.lemmatize(x,'v') for x in x.split()))

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from tqdm import tqdm

In [ ]:
print(test.shape)
print(train.shape)
test = all_data[all_data.Dataset == 'test']
print(test.shape)
train = all_data[all_data.Dataset == 'train']
print(train.shape)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 3),
    max_features=10000)

word_vectorizer.fit(all_data.question_text)
del all_data
train_word_features = word_vectorizer.transform(train.question_text)
test_word_features = word_vectorizer.transform(test.question_text)

In [ ]:
train_target = train['target'].values

kf = KFold(n_splits=5, shuffle=True, random_state=187)
test_pred_tf = 0
oof_pred_tf = np.zeros([train.shape[0],])

for i, (train_index, val_index) in tqdm(enumerate(kf.split(train))):
    x_train, x_val = train_word_features[train_index,:], train_word_features[val_index,:]
    y_train, y_val = train_target[train_index], train_target[val_index]
    classifier = LogisticRegression(class_weight = "balanced", C=0.5, solver='sag')
    classifier.fit(x_train, y_train)
    val_preds = classifier.predict_proba(x_val)[:,1]
    preds = classifier.predict_proba(test_word_features)[:,1]
    test_pred_tf += 0.2*preds
    oof_pred_tf[val_index] = val_preds

In [ ]:
import eli5
eli5.show_weights(classifier, vec=word_vectorizer, top=100, feature_filter=lambda x: x != '<BIAS>')

In [ ]:
import gensim
from gensim import corpora
#Creating a dictionary of words
dct = corpora.Dictionary(  nltk.word_tokenize(i) for i in  train.question_text )
train["question_text_tokens"] = list(map(nltk.word_tokenize, train.question_text))
# check threshold
dct.filter_extremes(no_below=20, no_above=0.5)
# Reindexes the remaining words after filtering
dct.compactify()
print("Left with {} words.".format(len(dct.values())))


In [ ]:
#Make a BOW for every document
def document_to_bow(df):
    df['bow'] = list(map(lambda doc: dct.doc2bow(doc), df.question_text_tokens))
    
document_to_bow(train)

## LDA Topic Model
# model imports
from gensim.models.ldamulticore import LdaMulticore
corpus = train.bow
num_topics = 100
#A multicore approach to decrease training time
LDAmodel = LdaMulticore(corpus=corpus,
                        id2word=dct,
                        num_topics=num_topics,
                        workers=4,
                        chunksize=4000,
                        passes=7,
                        alpha='asymmetric')
print("--- %s seconds for LDA ---" % (time.time() - start_time))

In [ ]:
def document_to_lda_features(lda_model, document):
    """ Transforms a bag of words document to features.
    It returns the proportion of how much each topic was
    present in the document.
    """
    topic_importances = LDAmodel.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]

train['lda_features'] = list(map(lambda doc:
                                      document_to_lda_features(LDAmodel, doc),
                                      train.bow))
print("--- %s seconds for LDA to Features ---" % (time.time() - start_time))

In [ ]:
Insincere_topic_distribution = train.loc[train.target == 1, 'lda_features'].mean()
Sincere_topic_distribution = train.loc[train.target == 0, 'lda_features'].mean()

In [ ]:
### Visualize Topics for different Classes
import matplotlib.pyplot as plt
fig, [ax1,ax2] = plt.subplots(2,1,figsize=(20,20))

nr_top_bars = 5

ax1.set_title("Insincere topic distributions", fontsize=16)
ax2.set_title("Sincere topic distributions", fontsize=16)

for ax, distribution, color in zip([ax1,ax2],
                                   [Insincere_topic_distribution,Sincere_topic_distribution],
                                   ['b','r']):
    # Individual distribution barplots
    ax.bar(range(len(distribution)), distribution, alpha=0.7)
    rects = ax.patches
    for i in np.argsort(distribution)[-nr_top_bars:]:
        rects[i].set_color(color)
        rects[i].set_alpha(1)
    # General plotting adjustments
    ax.set_xlim(-1, 150)
    ax.set_xticks(range(20,149,20))
    ax.set_xticklabels(range(20,149,20), fontsize=16)
    ax.set_ylim(0,0.02)
    ax.set_yticks([0,0.01,0.02])
    ax.set_yticklabels([0,0.01,0.02], fontsize=16)

fig.tight_layout(h_pad=3.)